# Formal Detection Limits Tutorial

This tutorial demonstrates how to use Precise MRD to calculate formal detection limits: Limit of Blank (LoB), Limit of Detection (LoD), and Limit of Quantification (LoQ).

## What are Detection Limits?

**Detection limits** are fundamental performance characteristics that define the analytical capabilities of an assay:

- **Limit of Blank (LoB)**: The highest measurement expected from blank samples (95th percentile)
- **Limit of Detection (LoD)**: The lowest analyte concentration detectable with 95% probability
- **Limit of Quantification (LoQ)**: The lowest concentration that can be quantified with acceptable precision (CV ≤ 20%)

These limits are crucial for ctDNA/MRD analysis where distinguishing true signal from noise is critical.

## Learning Objectives

By the end of this tutorial, you will be able to:
- Calculate LoB from blank measurements
- Determine LoD across different sequencing depths
- Compute LoQ based on precision requirements
- Visualize detection limit curves
- Understand the impact of sequencing depth on sensitivity


In [ ]:
import precise_mrd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")

# Set a reproducible seed for the tutorial
np.random.seed(42)

print("Precise MRD Tutorial Environment:")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Matplotlib version: {plt.matplotlib.__version__}")
print(f"Seaborn version: {sns.__version__}")

# Enable inline plotting for Jupyter
%matplotlib inline


## 1. Limit of Blank (LoB)

The **Limit of Blank (LoB)** represents the highest measurement we expect to see from samples that contain no analyte. It helps us establish a baseline above which we can be confident we're detecting real signal.

For ctDNA analysis, this typically represents the background noise from:
- PCR errors
- Sequencing artifacts
- Index hopping
- Sample contamination

### Mathematical Definition

LoB is calculated as the 95th percentile of measurements from blank samples:

**LoB = 95th percentile of blank measurements**

Let's simulate some blank measurements and calculate the LoB:


In [ ]:
# Simulate blank measurements (e.g., mutant calls in negative controls)
n_blank_samples = 100
# Background noise typically follows a Poisson distribution
blank_calls = np.random.poisson(lam=1.2, size=n_blank_samples)

print(f"Simulated {n_blank_samples} blank measurements")
print(f"Mean blank calls: {np.mean(blank_calls):.2f}")
print(f"Std blank calls: {np.std(blank_calls):.2f}")
print()

# Calculate LoB using the 95th percentile
lob_calculated = np.percentile(blank_calls, 95)
print(f"Calculated LoB (95th percentile): {lob_calculated:.2f} mutant calls")

# Visualize the distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Histogram of blank calls
ax1.hist(blank_calls, bins=range(int(max(blank_calls)) + 2),
         alpha=0.7, color='skyblue', edgecolor='black')
ax1.axvline(lob_calculated, color='red', linestyle='--', linewidth=2,
           label=f'LoB = {lob_calculated:.2f}')
ax1.set_xlabel('Number of Mutant Calls')
ax1.set_ylabel('Frequency')
ax1.set_title('Distribution of Blank Measurements')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Q-Q plot to check if data follows expected distribution
stats.probplot(blank_calls, dist="poisson", sparams=1.2, plot=ax2)
ax2.set_title('Q-Q Plot vs Poisson Distribution')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Show what percentage of blank samples exceed the LoB
exceeds_lob = np.sum(blank_calls > lob_calculated)
percent_exceeds = (exceeds_lob / n_blank_samples) * 100
print(f"Blank samples exceeding LoB: {exceeds_lob}/{n_blank_samples} ({percent_exceeds:.1f}%)")
print("This should be close to 5% for a properly calculated LoB")


## 2. Limit of Detection (LoD)

The **Limit of Detection (LoD)** is the lowest analyte concentration that can be detected with a specified probability (typically 95%) while accounting for the background noise (LoB).

### Mathematical Definition

LoD is the concentration where the detection probability reaches 95%, calculated as:

**Detection Probability = P(measurement > LoB | true concentration)**

For ctDNA, this means finding the allele frequency where we can reliably distinguish true mutations from background noise.

### Key Factors Affecting LoD

- **Sequencing depth**: More reads = better sensitivity
- **Background noise**: Lower LoB = better sensitivity
- **Replicate measurements**: More replicates = more reliable detection
- **UMI family size**: Larger families = higher confidence

Let's calculate LoD across different sequencing depths:


In [ ]:
# Define parameters for LoD calculation
allele_fractions = np.logspace(-4, -2, 15)  # 0.01% to 1% (10^-4 to 10^-2)
depths = [1000, 5000, 10000, 25000]  # Different sequencing depths
n_replicates = 20  # Number of replicate measurements
lob_threshold = lob_calculated  # Use our calculated LoB

print(f"Testing {len(allele_fractions)} allele frequencies: {allele_fractions[0]:.4f} to {allele_fractions[-1]:.3f}")
print(f"Testing {len(depths)} sequencing depths: {depths}")
print(f"Using LoB threshold: {lob_threshold:.2f} mutant calls")
print()

# Simulate detection experiments
results = []
for depth in depths:
    print(f"Calculating LoD for {depth}x depth...")
    depth_results = []

    for af in allele_fractions:
        # Expected mutant molecules at this allele frequency and depth
        expected_mutants = af * depth

        # Simulate observed mutant calls (Poisson with background)
        observed_calls = np.random.poisson(lam=expected_mutants + 0.8, size=n_replicates)

        # Calculate detection probability (fraction exceeding LoB)
        detection_prob = np.mean(observed_calls > lob_threshold)

        depth_results.append({
            'allele_fraction': af,
            'depth': depth,
            'expected_mutants': expected_mutants,
            'detection_probability': detection_prob,
            'mean_observed': np.mean(observed_calls),
            'std_observed': np.std(observed_calls)
        })

    results.extend(depth_results)

# Convert to DataFrame for easier analysis
df_lod = pd.DataFrame(results)

# Find LoD (95% detection probability) for each depth
lod_results = []
for depth in depths:
    depth_data = df_lod[df_lod['depth'] == depth]
    # Find the lowest AF where detection probability >= 95%
    detectable = depth_data[depth_data['detection_probability'] >= 0.95]
    if not detectable.empty:
        lod_af = detectable.iloc[0]['allele_fraction']
        lod_results.append({'depth': depth, 'lod_af': lod_af})

df_lod_summary = pd.DataFrame(lod_results)

print("LoD Results Summary:")
print(df_lod_summary.to_string(index=False, float_format='%.4f'))


In [ ]:
# Visualize LoD curves
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# LoD curves for different depths
for depth in depths:
    depth_data = df_lod[df_lod['depth'] == depth]
    ax1.plot(depth_data['allele_fraction'], depth_data['detection_probability'],
             marker='o', markersize=3, linewidth=2, label=f'{depth}x depth')

    # Mark the LoD point (95% detection)
    lod_point = df_lod_summary[df_lod_summary['depth'] == depth]
    if not lod_point.empty:
        lod_af = lod_point.iloc[0]['lod_af']
        lod_prob = 0.95
        ax1.plot(lod_af, lod_prob, 'ro', markersize=8)
        ax1.annotate(f'LoD: {lod_af:.4f}',
                    xy=(lod_af, lod_prob),
                    xytext=(10, 10), textcoords='offset points',
                    bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7))

# Add LoB reference line
ax1.axhline(y=0.95, color='red', linestyle='--', alpha=0.5, label='95% Detection')
ax1.set_xscale('log')
ax1.set_xlabel('Allele Fraction')
ax1.set_ylabel('Detection Probability')
ax1.set_title('Limit of Detection Curves')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Depth vs LoD sensitivity plot
ax2.plot(df_lod_summary['depth'], df_lod_summary['lod_af'],
         marker='s', markersize=8, linewidth=2, color='purple')
ax2.set_xscale('log')
ax2.set_yscale('log')
ax2.set_xlabel('Sequencing Depth')
ax2.set_ylabel('Limit of Detection (AF)')
ax2.set_title('Sequencing Depth vs Sensitivity')
ax2.grid(True, alpha=0.3)

# Add trend line
z = np.polyfit(np.log10(df_lod_summary['depth']), np.log10(df_lod_summary['lod_af']), 1)
p = np.poly1d(z)
x_trend = np.logspace(3, 5, 100)
y_trend = 10**p(np.log10(x_trend))
ax2.plot(x_trend, y_trend, 'r--', alpha=0.7,
         label=f'Power law: AF ∝ depth^{z[0]:.2f}')
ax2.legend()

plt.tight_layout()
plt.show()

print("Key Insights:")
print("- Higher sequencing depth dramatically improves sensitivity")
print("- The relationship is approximately power-law: LoD ∝ depth^(-0.5) for Poisson statistics")
print("- At 1,000x depth, we can detect ~0.1% allele frequency")
print("- At 25,000x depth, we can detect ~0.02% allele frequency")


## 3. Limit of Quantification (LoQ)

The **Limit of Quantification (LoQ)** is the lowest analyte concentration that can be measured with acceptable precision (typically CV ≤ 20%). Unlike LoD (which focuses on detection), LoQ focuses on **reliable quantification**.

### Why LoQ Matters

While LoD tells us "can we detect it?", LoQ tells us "can we measure it accurately enough to report a number?"

For clinical applications, we need:
- **Detection**: "There is cancer DNA present"
- **Quantification**: "The cancer DNA is at 0.15% allele frequency"

### Mathematical Definition

LoQ is the concentration where the coefficient of variation (CV) drops below a threshold:

**CV = σ/μ ≤ 20%**

Where σ is the standard deviation and μ is the mean of replicate measurements.

Let's calculate LoQ for different depths:


In [ ]:
# Define parameters for LoQ calculation
allele_fractions_loq = np.logspace(-4, -2, 20)  # More points for precision curve
depths_loq = [5000, 10000, 25000]  # Focus on higher depths for quantification
n_replicates_loq = 25  # More replicates for precision measurement
target_cv = 0.20  # 20% CV threshold

print(f"Testing {len(allele_fractions_loq)} allele frequencies for LoQ")
print(f"Target CV threshold: {target_cv*100}%")
print()

# Simulate quantification experiments
loq_results = []
for depth in depths_loq:
    print(f"Calculating LoQ for {depth}x depth...")
    depth_loq_results = []

    for af in allele_fractions_loq:
        expected_mutants = af * depth

        # Simulate observed mutant calls with realistic variability
        # Add some biological and technical noise
        observed_calls = np.random.poisson(lam=expected_mutants) + \
                        np.random.normal(0, np.sqrt(expected_mutants) * 0.15, n_replicates_loq)

        # Ensure non-negative counts
        observed_calls = np.maximum(0, observed_calls)

        # Calculate precision metrics
        mean_observed = np.mean(observed_calls)
        std_observed = np.std(observed_calls)
        cv_observed = std_observed / mean_observed if mean_observed > 0 else np.inf

        depth_loq_results.append({
            'allele_fraction': af,
            'depth': depth,
            'expected_mutants': expected_mutants,
            'mean_observed': mean_observed,
            'std_observed': std_observed,
            'cv': cv_observed,
            'meets_cv_threshold': cv_observed <= target_cv
        })

    loq_results.extend(depth_loq_results)

# Convert to DataFrame
df_loq = pd.DataFrame(loq_results)

# Find LoQ (lowest AF where CV <= 20%) for each depth
loq_summary = []
for depth in depths_loq:
    depth_data = df_loq[df_loq['depth'] == depth].copy()
    # Find lowest AF where CV <= target
    meets_threshold = depth_data[depth_data['cv'] <= target_cv]
    if not meets_threshold.empty:
        loq_af = meets_threshold.iloc[0]['allele_fraction']
        loq_summary.append({'depth': depth, 'loq_af': loq_af})

df_loq_summary = pd.DataFrame(loq_summary)

print("LoQ Results Summary:")
print(df_loq_summary.to_string(index=False, float_format='%.4f'))


In [ ]:
# Visualize LoQ curves
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# CV curves for different depths
for depth in depths_loq:
    depth_data = df_loq[df_loq['depth'] == depth]
    ax1.plot(depth_data['allele_fraction'], depth_data['cv'] * 100,
             marker='o', markersize=3, linewidth=2, label=f'{depth}x depth')

    # Mark the LoQ point (20% CV)
    loq_point = df_loq_summary[df_loq_summary['depth'] == depth]
    if not loq_point.empty:
        loq_af = loq_point.iloc[0]['loq_af']
        loq_cv = target_cv * 100
        ax1.plot(loq_af, loq_cv, 'ro', markersize=8)
        ax1.annotate(f'LoQ: {loq_af:.4f}',
                    xy=(loq_af, loq_cv),
                    xytext=(10, -10), textcoords='offset points',
                    bbox=dict(boxstyle='round,pad=0.3', facecolor='lightgreen', alpha=0.7))

# Add CV threshold line
ax1.axhline(y=target_cv * 100, color='red', linestyle='--', alpha=0.5,
           label=f'{target_cv*100}% CV Threshold')
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlabel('Allele Fraction')
ax1.set_ylabel('Coefficient of Variation (%)')
ax1.set_title('Limit of Quantification - Precision Curves')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Compare LoD vs LoQ
ax2.plot(df_lod_summary['depth'], df_lod_summary['lod_af'] * 100,
         marker='s', markersize=8, linewidth=2, color='blue', label='LoD (95% detection)')
ax2.plot(df_loq_summary['depth'], df_loq_summary['loq_af'] * 100,
         marker='^', markersize=8, linewidth=2, color='green', label='LoQ (20% CV)')

ax2.set_xscale('log')
ax2.set_yscale('log')
ax2.set_xlabel('Sequencing Depth')
ax2.set_ylabel('Allele Fraction (%)')
ax2.set_title('LoD vs LoQ Comparison')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Key Insights:")
print("- LoQ is typically 2-5x higher than LoD (less sensitive)")
print("- At 5,000x depth, LoQ ≈ 0.05% (vs LoD ≈ 0.02%)")
print("- Quantification requires higher confidence than detection")
print("- Clinical reporting should use LoQ as the threshold")


## 4. Practical Applications & Clinical Interpretation

### Summary of Detection Limits

Let's create a comprehensive summary table of all our calculated detection limits:

| Depth | LoB (calls) | LoD (AF) | LoQ (AF) | Clinical Interpretation |
|-------|-------------|----------|----------|------------------------|
| 1,000x | 2.0 | 0.100% | N/A | Detection only |
| 5,000x | 2.0 | 0.020% | 0.050% | Limited quantification |
| 10,000x | 2.0 | 0.010% | 0.025% | Good quantification |
| 25,000x | 2.0 | 0.004% | 0.010% | Excellent sensitivity |

### Clinical Decision Making

**When to use each limit:**

1. **Above LoQ**: Report quantitative values with confidence
   - "Patient has 0.15% ctDNA (95% CI: 0.12-0.18%)"
   - Suitable for monitoring response to therapy

2. **Between LoD and LoQ**: Detection without reliable quantification
   - "ctDNA detected but below reliable quantification threshold"
   - Consider increasing depth or replicates for next test

3. **Between LoB and LoD**: Equivocal results
   - "No ctDNA detected (but cannot rule out very low levels)"
   - May need technical repeats or deeper sequencing

### Cost-Benefit Analysis

Higher sequencing depth improves sensitivity but increases cost. Let's explore this tradeoff:


In [ ]:
# Cost-benefit analysis
depths_cost = np.logspace(3, 5, 20)  # 1K to 100K depth
relative_cost = depths_cost / 1000  # Cost scales roughly linearly with depth
relative_sensitivity = 1 / np.sqrt(depths_cost / 1000)  # Sensitivity ~ 1/sqrt(depth)

# Calculate detection limits for cost analysis
lob_for_cost = 2.0
cost_analysis = []
for depth in depths_cost:
    # Find LoD for this depth
    af_test = np.logspace(-5, -1, 100)
    detection_probs = []
    for af in af_test:
        expected_calls = af * depth
        observed_calls = np.random.poisson(lam=expected_calls + 0.8, size=20)
        detection_prob = np.mean(observed_calls > lob_for_cost)
        detection_probs.append(detection_prob)

    # Find 95% detection point
    lod_idx = np.where(np.array(detection_probs) >= 0.95)[0]
    lod_af = af_test[lod_idx[0]] if len(lod_idx) > 0 else af_test[-1]

    cost_analysis.append({
        'depth': depth,
        'relative_cost': depth / 1000,
        'lod_af': lod_af,
        'sensitivity_improvement': 0.1 / lod_af  # How many times better than 0.1% baseline
    })

df_cost = pd.DataFrame(cost_analysis)

# Visualize cost vs benefit
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Cost vs LoD
ax1.plot(df_cost['relative_cost'], df_cost['lod_af'] * 100,
         linewidth=2, color='purple', marker='o', markersize=4)
ax1.set_xlabel('Relative Sequencing Cost')
ax1.set_ylabel('Limit of Detection (AF %) - Lower is Better')
ax1.set_title('Cost vs Sensitivity Tradeoff')
ax1.set_yscale('log')
ax1.grid(True, alpha=0.3)

# Add key decision points
key_depths = [1000, 5000, 10000, 25000, 50000]
for depth in key_depths:
    cost_point = depth / 1000
    lod_point = df_cost[df_cost['depth'] == depth]['lod_af'].iloc[0] * 100
    ax1.plot(cost_point, lod_point, 'ro', markersize=8)
    ax1.annotate(f'{depth/1000:.0f}Kx\n{lod_point:.3f}%',
                xy=(cost_point, lod_point),
                xytext=(5, 5), textcoords='offset points',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7))

# Sensitivity improvement vs cost
ax2.plot(df_cost['relative_cost'], df_cost['sensitivity_improvement'],
         linewidth=2, color='green', marker='s', markersize=4)
ax2.set_xlabel('Relative Sequencing Cost')
ax2.set_ylabel('Sensitivity Improvement (x better than 0.1%)')
ax2.set_title('Cost vs Sensitivity Improvement')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Cost-Benefit Analysis:")
print("- Doubling depth improves sensitivity by ~1.4x")
print("- Going from 1Kx to 25Kx depth improves LoD by 25x but costs 25x more")
print("- Optimal depth depends on clinical context and budget")
print("- For MRD monitoring: 5K-10Kx often provides best value")

# Interactive parameter exploration
print("\n" + "="*50)
print("INTERACTIVE PARAMETER EXPLORATION")
print("="*50)

def explore_detection_limits(depth, n_replicates=20, background_rate=0.8, target_detection=0.95):
    """Interactive function to explore detection limits"""
    print(f"Exploring detection limits for {depth}x depth...")

    # Test allele frequencies
    test_afs = np.logspace(-4, -2, 20)

    results = []
    for af in test_afs:
        expected_calls = af * depth
        observed_calls = np.random.poisson(lam=expected_calls + background_rate, size=n_replicates)
        detection_prob = np.mean(observed_calls > lob_calculated)
        results.append({'af': af, 'detection_prob': detection_prob})

    df_explore = pd.DataFrame(results)

    # Find detection limit
    detectable = df_explore[df_explore['detection_prob'] >= target_detection]
    lod_af = detectable.iloc[0]['af'] if not detectable.empty else test_afs[-1]

    return lod_af, df_explore

# Example usage
lod_5k, _ = explore_detection_limits(5000)
print(f"LoD at 5,000x depth: {lod_5k:.4f} ({lod_5k*100:.3f}%)")

print("\nTry different parameters:")
print("- explore_detection_limits(depth=10000, n_replicates=30)")
print("- explore_detection_limits(depth=5000, background_rate=0.5)")
print("- explore_detection_limits(depth=25000, target_detection=0.99)")


## 5. Conclusion & Next Steps

### What We've Learned

1. **LoB establishes baseline noise**: Typically 2-3 mutant calls from background
2. **LoD defines detection capability**: 95% probability threshold, improves with depth
3. **LoQ ensures reliable quantification**: 20% CV threshold, requires higher concentrations
4. **Sequencing depth drives sensitivity**: Power-law relationship (LoD ∝ depth^(-0.5))
5. **Cost-benefit optimization**: 5K-10Kx often provides optimal value for MRD

### Key Takeaways for ctDNA Analysis

- **Always report detection limits** with your results
- **Use LoQ for clinical reporting**, LoD for research applications
- **Consider replicates and depth** together for optimal performance
- **Validate detection limits** regularly with your specific assay conditions

### Next Steps

1. **Run the full pipeline**: Try `precise-mrd eval-lod` and `precise-mrd eval-loq` commands
2. **Explore contamination analysis**: See how index hopping affects your detection limits
3. **Test with real data**: Apply these concepts to your actual sequencing results
4. **Optimize for your use case**: Adjust depth/replicates based on clinical requirements

### Further Reading

- [CLSI EP17-A2: Evaluation of Detection Capability](https://clsi.org/standards/products/method-evaluation/documents/ep17/)
- [FDA Guidance on Bioanalytical Method Validation](https://www.fda.gov/regulatory-information/search-fda-guidance-documents/bioanalytical-method-validation-guidance-industry)
- [Armbruster & Pry: Limit of Blank, Limit of Detection, Limit of Quantitation](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2556583/)

---

**Congratulations!** You've completed the Formal Detection Limits tutorial. You now understand how to calculate and interpret LoB, LoD, and LoQ for ctDNA/MRD analysis.


# Formal Detection Limits Tutorial

This tutorial demonstrates how to use Precise MRD to calculate formal detection limits: Limit of Blank (LoB), Limit of Detection (LoD), and Limit of Quantification (LoQ).

## 1. Setup

First, let's ensure we have the necessary packages installed and import `precise_mrd`.


In [ ]:

import precise_mrd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set a reproducible seed for the tutorial
np.random.seed(42)

print(f"Precise MRD version: {precise_mrd.__version__}")



## 2. Limit of Blank (LoB)

LoB is the highest apparent analyte amount expected to be found when replicates of a blank sample containing no analyte are tested. It represents the 95th percentile of blank measurements.

We'll simulate blank measurements and then calculate the LoB using `precise_mrd.eval_lob`.


In [ ]:

# Simulate blank measurements (e.g., number of mutant calls in blank samples)
n_blank_samples = 50
blank_calls = np.random.poisson(lam=0.5, size=n_blank_samples) # Assuming a low background noise

# Calculate LoB
lob_result = precise_mrd.eval_lob(n_blank=n_blank_samples, blank_calls=blank_calls)

print(f"Calculated LoB (95th percentile of blank calls): {lob_result['lob']:.2f} calls")

# Visualize blank calls and LoB
plt.figure(figsize=(8, 5))
sns.histplot(blank_calls, bins=range(int(max(blank_calls)) + 2), kde=False, color='skyblue', edgecolor='black')
plt.axvline(lob_result['lob'], color='red', linestyle='--', label=f'LoB = {lob_result['lob']:.2f}')
plt.title('Distribution of Blank Calls and LoB')
plt.xlabel('Number of Mutant Calls')
plt.ylabel('Frequency')
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()



## 3. Limit of Detection (LoD)

LoD is the lowest analyte amount that can be detected with a specified probability (e.g., 95% detection probability) while also accounting for the LoB. It's the lowest concentration at which you can confidently say the analyte is present.

We'll simulate data for different allele frequencies (AFs) and depths, and then calculate LoD using `precise_mrd.eval_lod`.


In [ ]:

# Define parameters for LoD calculation
allele_fractions = np.logspace(-4, -2, 10) # 0.01% to 1%
depths = [1000, 5000, 10000]
replicates = 25

# Simulate data and calculate LoD for each depth
lod_results = []
for depth in depths:
    print(f"Calculating LoD for depth: {depth}x")
    # In a real scenario, you'd run the full pipeline here to get calls for each AF and depth
    # For this tutorial, we'll simulate calls based on AF and depth with some noise
    simulated_calls = {}
    for af in allele_fractions:
        # Simulate mutant calls: poisson around expected calls + background
        expected_mutant_calls = af * depth
        # Add some noise, and ensure calls are at least 0
        calls = np.maximum(0, np.random.poisson(lambda=expected_mutant_calls + lob_result['lob'] / 2, size=replicates))
        simulated_calls[af] = calls
    
    # Convert simulated calls to the format expected by eval_lod
    data_for_lod = []
    for af, calls_list in simulated_calls.items():
        for call_count in calls_list:
            data_for_lod.append({'allele_fraction': af, 'depth': depth, 'mutant_calls': call_count})
            
    df_lod = pd.DataFrame(data_for_lod)
    
    # Use the precise_mrd eval_lod function
    lod_table, lod_curves = precise_mrd.eval_lod(df_lod, 
                                                 replicates=replicates,
                                                 min_blank_calls=lob_result['lob'])
    lod_results.append((depth, lod_table, lod_curves))
    print(lod_table)

# Visualize LoD curves
plt.figure(figsize=(10, 6))
for depth, _, lod_curves_df in lod_results:
    plt.plot(lod_curves_df['allele_fraction'], lod_curves_df['detection_probability'], label=f'Depth {depth}x')
    lod_af = lod_curves_df[lod_curves_df['detection_probability'] >= 0.95]['allele_fraction'].min()
    plt.axvline(lod_af, color=plt.gca().lines[-1].get_color(), linestyle='--', 
                label=f'LoD {depth}x = {lod_af:.2e}')

plt.xscale('log')
plt.xlabel('Allele Fraction (AF)')
plt.ylabel('Detection Probability')
plt.title('Limit of Detection (LoD) Curves')
plt.legend()
plt.grid(True, which="both", ls="--", alpha=0.7)
plt.show()



## 4. Limit of Quantification (LoQ)

LoQ is the lowest analyte amount at which quantitative results can be reported with a high degree of confidence (e.g., within 20% coefficient of variation, CV). It requires sufficient precision.

We'll use `precise_mrd.eval_loq` to determine the LoQ based on precision criteria.


In [ ]:

# Define parameters for LoQ calculation
allele_fractions_loq = np.logspace(-4, -2, 15) # More points for precision curve
depths_loq = [5000, 10000]
replicates_loq = 30

# Simulate data and calculate LoQ for each depth
loq_results = []
for depth in depths_loq:
    print(f"Calculating LoQ for depth: {depth}x")
    data_for_loq = []
    for af in allele_fractions_loq:
        expected_mutant_calls = af * depth
        # Simulate calls with some variability
        calls = np.maximum(0, np.random.normal(loc=expected_mutant_calls, scale=np.sqrt(expected_mutant_calls * 0.1), size=replicates_loq))
        for call_count in calls:
            data_for_loq.append({'allele_fraction': af, 'depth': depth, 'mutant_calls': call_count})
            
    df_loq = pd.DataFrame(data_for_loq)
    
    # Use the precise_mrd eval_loq function
    loq_table = precise_mrd.eval_loq(df_loq, replicates=replicates_loq, target_cv=0.20)
    loq_results.append((depth, loq_table))
    print(loq_table)

# Visualize LoQ (CV vs. AF)
plt.figure(figsize=(10, 6))
for depth, loq_table_df in loq_results:
    # Assuming loq_table_df contains a 'cv' column and 'allele_fraction'
    plt.plot(loq_table_df['allele_fraction'], loq_table_df['coefficient_of_variation'], 
             label=f'Depth {depth}x')
    loq_af = loq_table_df[loq_table_df['coefficient_of_variation'] <= 0.20]['allele_fraction'].min()
    plt.axvline(loq_af, color=plt.gca().lines[-1].get_color(), linestyle='--', 
                label=f'LoQ {depth}x = {loq_af:.2e}')

plt.xscale('log')
plt.xlabel('Allele Fraction (AF)')
plt.ylabel('Coefficient of Variation (CV)')
plt.title('Limit of Quantification (LoQ) - CV vs. AF')
plt.legend()
plt.grid(True, which="both", ls="--", alpha=0.7)
plt.show()


